# Internet of Things

El microcontrolador que estamos usando es NodeMCU v1.0. ESP-12E.

Al encender el microcontrolador se carga el boot.py y luego el main.py.

Casi todos los main.py van a trabajar con un bucle while.

## NUNCA CONECTAR POSITIVO Y NEGATIVO CON UN CABLE!!!!!!!!

## Hello world:

In [ ]:
import machine
import utime

led = machine.Pin(2, machine.Pin.OUT)

while True:
    led.on()
    utime.sleep(1)
    led.off()
    utime.sleep(1)

## Estado del botón:

In [ ]:
import machine
import utime
led = machine.Pin(2, machine.Pin.OUT)
boton = machine.Pin(0, machine.Pin.IN)
while True:

    estado_boton = boton.value()
    print(estado_boton)
    led.value(estado_boton)
    utime.sleep(0.1)

## Pull.UP para definir voltaje a 0 y 1

In [ ]:
from machine import Pin
import utime

boton2 = Pin(16, Pin.PULL_UP)

while True:
    valor = boton2.value()
    print('valor: {}'.format(valor))
    utime.sleep(0.1)

# Hay que conectar un cable dupont al pin 16 y a tierra hace que el value sea 0

## Recreación salida analógica con una salida digital

Usamos PWM para dar una salida analógica mediante una digital. Nos permites más estados que 0 y 1.

In [ ]:
from machine import Pin, PWM
import utime

led = Pin(2, Pin.OUT)
led_pwm = PWM(led) # la frequencia se puede especificar ahí como segundo elemento de la tupla o por separado

led_pwm.freq(500)

while True:
    for i in range(1023):
        led_pwm.duty(i)
        utime.sleep_ms(3)
    for i in range(1023, 0, -1):
        led_pwm.duty(i)
        utime.sleep_ms(3)

Con PWM podemos usar servos y ccntrolar así el movimiento que hace, siendo `0` y `1` los valores límite y los intermedios, posiciones intermedias.

Podemos hacer lo mismo pero con un potenciómetro (que no tenemos) y depende del voltaje, da más intensidad o menos.

In [ ]:
from machine import Pin, PWM, ADC
import utime

led = Pin(2, Pin.OUT)
led_pwm = PWM(led)
led_pwm.freq(1000)

pot = ADC(0)

while True:
    lectura = pot.read()
    led_pwm.duty(lectura)
    print(lectura)
    utime.sleep(0.5)

## Los Timers:

Se usan con `machine.Timer()` y se usan para tareas recurrentes o para hacer contadores.

Con `Timer(-1)` tenemos un timer simulado por software y con otros números los de hardware.

Iniciamos con .init() y los parámetros son:
- `period`: El periodo, cada cuánto se activa
- `mode`: Tenemos el `ONE_SHOT`, que se activa una vez y el `PERIODIC`, que se activa cada los milisegundos indicados.
- `callback`: Podemos especificar un callback, usando funciones (incluyendo lambdas).

In [ ]:
from machine import Pin, PWM, ADC
import utime

tim = Timer(-1)

def texto(x): # x es el lugar en memoria de la instancia
    print('hola') # nunca poner un sleep aquí, ya que se romperá con el period de abajo

tim.init(period=1000, mode=Timer.PERIODIC, callback=texto)

Podemos usar `time.deinit()` para parar el timer. Si hacemos un sleep antes, el timer se va a estar ejecutando durante ese tiempo, lo que nos permite controlar las veces que se ejecuta.

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

tim = Timer(-1)

def texto(x):
    print('hola')

tim.init(period=1000, mode=Timer.PERIODIC, callback=texto)

utime.sleep(3)

tim.deinit()

La técnica usada anteriormente para leer el botón se llama pulling. Lee el estado periódicamente del botón. No funciona bien con sleeps muy altos.

Podemos usar un timer para lanzar un callback cuando se pulse. Para ello se usa el método `.irq()`, que tiene dos parámetros:
- `función`: La función que debe ejecutar
- `trigger`: Define lo que hará que se lance la señal.
    - `IRQ_FALLING`: Pulsación del botón
    - `IRQ_RISING`: Soltar el botón

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

def boton_callback(x):
    print('botón pulsado')

boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING) # tráfico descendente

while True:
    estado = boton.value()
    print('x')
    utime.sleep(0.1)

Otra implementación:

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

boton_pulsado= False

def boton_callback(x):
    global boton_pulsado
    boton_pulsado = True

boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING) # tráfico descendente

while True:
    print('x')
    if boton_pulsado:
        print("boton_pulsado")
        boton_pulsado = False
    utime.sleep(0.1)

En el mundo real hay un montón de ruido al pulsar el botón y no es tan fácil capturar el raising y el falling. Se puede lidiar con ese problema de forma manual o usando librerías específicas. Usando el código anterior, nos metemos en el bucle:

In [ ]:
from machine import Pin, PWM, ADC, Timer
import utime

boton_pulsado = False
tiempo_inicio_boton = 0

def boton_callback(x):
    global boton_pulsado, tiempo_inicio_boton
    boton_pulsado = True
    tiempo_inicio_boton = utime.ticks_ms()    


boton = Pin(0, Pin.IN)
boton.irq(boton_callback, trigger=Pin.IRQ_FALLING)

while True:
    print("x")
    if boton_pulsado:
        print("boton pulsado marca {}".format(tiempo_inicio_boton))
        ahora =  utime.ticks_ms()  
        tiempo_boton = utime.ticks_diff(ahora, tiempo_inicio_boton)

        print(tiempo_boton)
        if tiempo_boton < 50:
            utime.sleep_ms(50-tiempo_boton)
        estado = boton.value()
        if estado:
            print("esto es ruido, no hacer nada")
        else:
            print("boton pulsado de verdad")
        utime.sleep(0.5) #antirrebotes
        boton_pulsado = False
    utime.sleep(0.1)

Para más implementaciones: http://docs.micropython.org/en/v1.9.3/pyboard/pyboard/tutorial/debounce.html

### Otras librerías interesantes:
- `machine`
- `utime`
- `esp`

In [ ]:
from machine import Pin
from utime import sleep

class Boton:
    def __init__(self, pin_n):
        self.boton = Pin(pin_n, Pin.IN) # PULL_UP
        self.boton.irq(self.pulsado_cb, Pin.IRQ_FALLING)
        self.pulsado = False

    def pulsado_cb(self, inst):
        self.pulsado = True

    def get_pulsado(self):
        if not self.pulsado:
            return False
        self.pulsado = False
        return True



boton = Boton(0)


while True:
    if boton.get_pulsado():
        print('se ha pulsado el boton')
    print('x', end="")
    sleep(0.5)

Mismo código pero con las ayudas para detectar bien el botón

In [ ]:
from machine import Pin
from utime import sleep
import micropython

class Boton:
    def __init__(self, pin_n):
        self.boton = Pin(pin_n, Pin.IN) # PULL_UP
        self.boton.irq(self.pulsado_cb, Pin.IRQ_FALLING)
        self.pulsado = False

    def pulsado_cb(self, inst):
        micropython.schedule(self.post_pulsado_cb, 0)

    def post_pulsado_cb(self, inst):
        # aqui puedo hacer operaciones más caras, ya que lo hace el rtOS
        # self.pulsado = True
        self.boton.irq(self.liberado_cb, Pin.IRQ_RISING)

    def liberado_cb(self, inst):
        micropython.schedule(self.post_liberado_cb, 0)

    def post_liberado_cb(self, inst):
        self.pulsado = True
        self.boton.irq(self.pulsado_cb, Pin.IRQ_FALLING)

    def get_pulsado(self):
        if not self.pulsado:
            return False
        self.pulsado = False
        return True



boton = Boton(0)


while True:
    if boton.get_pulsado():
        print('\nse ha pulsado el boton')
    print('x', end="")
    sleep(0.5)


# Redes

In [ ]:
import network

red = network.WLAN(network.AP_IF)
red.active(True)
red.config(essid="MicroPython", password="12345678")

##  AP con web para controlar el led

In [ ]:
import network
import usocket as socket
from machine import Pin

def web_page(estado):
    html = """<html><head> <title>Ejemplo 1</title> <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="icon" href="data:;base64,iVBORw0KGgo="><style>html{font-family: Helvetica; display:inline-block; margin: 0px auto; text-align: center;}
    h1{color: #0F3376; padding: 2vh;}p{font-size: 1.5rem;}.button{display: inline-block; background-color: #e7bd3b; border: none;
    border-radius: 4px; color: white; padding: 16px 40px; text-decoration: none; font-size: 30px; margin: 2px; cursor: pointer;}
    .button2{background-color: #4286f4;}</style></head><body> <h1>CONTROL DE LED</h1>
    <p>Estado LED: <strong> """ + estado + """ </strong></p><p><a href="/?led=on"><button class="button">ON</button></a></p>
    <p><a href="/?led=off"><button class="button button2">OFF</button></a></p></body></html>"""
    return html

led = Pin(2, Pin.OUT)
led.value(1)

red = network.WLAN(network.AP_IF)
red.active(True)
red.config(essid="MicroPython", password="12345678")

print(red.ifconfig())

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(('0.0.0.0', 80))
sock.listen(5)
# sock.settimeout(30)

while True:
    print('Esperando conexion...')
    conn, addr = sock.accept()
    print(addr)
    request = conn.recv(1024)
    request_str = request.decode()
    print(request_str)
    respuesta = '<h1>Hola, mundo</h1> yo soy Rober'
    ledon = request_str.find("/?led=on")
    print(ledon)
    ledoff = request_str.find("/?led=off")
    print(ledoff)

    estado_led = led.value()
    estado_str = "ENCENDIDO" if not estado_led else "APAGADO"

    if  ledon == 4:
        led.value(0)
        respuesta = "Led encendido"
    elif ledoff == 4:
        respuesta = "Led apagado"
        led.value(1)

    conn.sendall('HTTP/1.1 200 OK\n')
    conn.sendall('Content-Type: text/html\n')
    conn.sendall('Connection: close\n\n')
    conn.sendall(web_page(estado_str))
    conn.close()


## Conexión a tu wifi

In [ ]:
from machine import Pin
import network
import utime as time

from credenciales import ssid, password # hay que crear el fichero credenciales


# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)

print('conectado!')
print(red.ifconfig())
led.value(1)

In [1]:
# Fichero credenciales

ssid = "wifi"
password = "contraseña"

## Misma web para encender o apagar led

In [ ]:
from machine import Pin
import network
import utime as time

from credenciales import ssid, password # hay que crear el fichero credenciales
from corneto import Corneto

# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)

print('conectado!')
print(red.ifconfig())
led.value(1)

web = Corneto()

def home(x):
    contexto = {
        "tiempo": str(time.ticks_ms() // 1000),

    }
    return("index.html", contexto)

def config(x):
    contexto = {}
    return("configuracion.html", contexto)

def led_encender(x):
    led.value(0)
    contexto = {}
    return("luz.html", contexto)

def led_apagar(x):
    led.value(1)
    contexto = {}
    return("luz.html", contexto)

web.add_view('/', home)
web.add_view('/configuracion', config)
web.add_view('/encender', led_encender)
web.add_view('/apagar', led_apagar)


web.run_server()

## Programa para enviar mensaje por UDP

In [ ]:
# Desde el microcontrolador

from machine import Pin
import network
import utime as time
import usocket as socket

from credenciales import ssid, password # hay que crear el fichero credenciales

# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose 2a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

while True:
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.settimeout(3)
    mensaje = b"Hola, mundo!"
    print(mensaje)
    sock.sendto(mensaje, ("192.168.1.255", 5005))
    # sock.setsockopt(SOL_SOCKET, SO_BROADCAST, 1) # esto en usocket no funciona bien
    try:
        data, addr = sock.recvfrom(512)
        print(addr)
        print(data)
    except OSError: # TIMEOUT
        print("no hay respuesta")
    sock.close()
    led.value(0)
    time.sleep(0.1)
    led.value(1)
    time.sleep(1)

In [2]:
# Desde servidor

import socket

UDP_PORT = 5005

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(("0.0.0.0", UDP_PORT))
print("Escuchando por el puerto {}".format(UDP_PORT))

try:
    while True:
        
        try:
            data, addr = sock.recvfrom(512)

        except socket.timeout:
            continue

        print(addr)
        print(data)
        sock.sendto(b'Te he oido', addr)

except KeyboardInterrupt:
    sock.close()

Escuchando por el puerto 5005
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'
('192.168.1.115', 4097)
b'Hola, mundo!'


## Usar `ntptime` para obtener el datetime

In [ ]:
# el schedule nos da problemas

from machine import Pin, RTC
import network
import ntptime
from micropython import schedule
import utime as time
import usocket as socket

from credenciales import ssid, password # hay que crear el fichero credenciales

# Configurar hardware
led = Pin(2, Pin.OUT)


# Conexión wifi
led.value(0)
print('\nConectandose 2a wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected(): # Espera hasta que conecte
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

rtc = RTC()
rtc.datetime((2020,6,15,19,2,0,0,0)) # configuración de hora manual

ntptime.settime()
rtc.datetime()
print(rtc.datetime())

def texto_cb():
    print(rtc.datetime())

schedule.every(2).minutes.do(texto_cb)

while True:
    schedule.run_pending()
    time.sleep(1)

## Instalar urequests

In [ ]:
from machine import Pin
import utime as time
import network


from credenciales import ssid, password


led = Pin(2, Pin.OUT)


led.value(0)
print('\nConectandose a la wifi...', end='')
red = network.WLAN(network.STA_IF)
red.active(True)
red.connect(ssid, password)
while not red.isconnected():
    time.sleep(0.1)
print('conectado!')
print(red.ifconfig())
led.value(1)

import upip
upip.install('micropython-urequests')

## Llamadas `API` usando JSON